In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from scipy.stats import pearsonr, spearmanr, shapiro
import plotly.graph_objects as go
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, Dropdown
from IPython.display import display

## Pre-processing

In [2]:
# Import data
df_c40 = pd.read_excel('./C40_GPC_Database.xlsx', sheet_name='GHG Dashboard Data - Inventory')

In [3]:
# select the interested columns
df_c40 = df_c40[['City','Country','Region','Boundary','Year_calendar','Population','Area','GDP','I.1.1','I.1.2','I.1.3','I.2.1',
                 'I.2.2','I.2.3','I.3.1','I.3.2','I.3.3','I.4.1','I.4.2','I.4.3','I.4.4','I.5.1','I.5.2','I.5.3','I.6.1',
                 'I.6.2','I.6.3','I.7.1','I.8.1','II.1.1','II.1.2','II.1.3','II.2.1','II.2.2','II.2.3','II.3.1','II.3.2',
                 'II.3.3','II.4.1','II.4.2','II.4.3','II.5.1','II.5.2','II.5.3','III.1.1','III.1.2','III.1.3','III.2.1',
                 'III.2.2','III.2.3','III.3.1','III.3.2','III.3.3','III.4.1','III.4.2','III.4.3','IV.1','IV.2','V.1','V.2',
                 'V.3','VI.1']]

In [4]:
# re-format the data
df_c40 = pd.melt(df_c40, 
                 id_vars=['City','Country','Region','Boundary','Year_calendar','Population','Area','GDP'], 
                 var_name='gpc_refno', 
                 value_name='emissions')

In [5]:
# standarize the notation keys
df_c40 = df_c40.replace({'IE (I.1.1)': 'IE', 'NO; IE': 'NO', 'NE; NE': 'NE', 'IE (II.1.1)': 'IE', 'NO ': 'NO'})

In [6]:
def is_non_numeric(value):
    try:
        float(value)
        return False  # Numeric values
    except ValueError:
        return True   # Non-numeric values

In [7]:
df = df_c40.copy()
# keeping only numeric values
df['emissions'] = pd.to_numeric(df['emissions'], errors='coerce')

In [8]:
# Filter for negative emissions
negative_values_mask = df['emissions'] < 0

negative_emissions_df = df[negative_values_mask]

In [9]:
# drop negative values and NAN values
clean_df = df[~negative_values_mask]
clean_df = clean_df.dropna(subset=['emissions'])

# replace a non-numeric value in the GDP column
clean_df['GDP'] = clean_df['GDP'].replace({'$763,955,224,976': 763955224976})

/var/folders/v8/b0fnjf9d4q31sfz_8q4h4zxm0000gn/T/ipykernel_63745/3103458883.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clean_df['GDP'] = clean_df['GDP'].replace({'$763,955,224,976': 763955224976})


In [10]:
# deleting values = 0
clean_df = clean_df[clean_df['emissions'] != 0]
clean_df = clean_df[clean_df['GDP'] != 0]


In [11]:
clean_df['emissions'] *= 1000
clean_df['units'] = 'kg'
clean_df['gas'] = 'CO2e'

In [12]:
# Calculate the log of emissions
clean_df['log_emissions'] = np.log(clean_df['emissions']+1)

In [13]:
clean_df_reformat = clean_df.melt(
    id_vars=['City', 'Country', 'Region', 'Boundary', 'Year_calendar', 'gpc_refno', 'emissions', 'units', 'gas'],
    value_vars=['Population', 'Area', 'GDP'],
    var_name='Variable_name',
    value_name='Variable_value'
)

In [16]:
#clean_df_reformat.to_csv('clean_c40.csv', index=False)

## Normal Distribution analysis

**Check Normality of "Emissions" by subsector and year**

In [17]:
def shapiro_test(df, group_col, value_col):
    """
    Perform the Shapiro-Wilk test for normality.
    """

    # Suppress warnings for small sample sizes
    warnings.filterwarnings("ignore")

    shapiro_results = []

    grouped = df.groupby([f'{group_col}', 'Year_calendar'])

    for subsector, group in grouped:
        tmp = group[f'{value_col}'].values
        n = len(tmp)
        
        # Shapiro-Wilk Test requires 25 < n < 5000
        if n < 25:
            shapiro_stat, shapiro_p = np.nan, np.nan
            note = "Insufficient data"
        else:
            shapiro_stat, shapiro_p = shapiro(tmp)
            note = ""
        
        # Append the results
        shapiro_results.append({
            f'{group_col}': subsector,
            'Shapiro_Wilk_Statistic': shapiro_stat,
            'p-value': shapiro_p,
            'Sample_Size': n,
            'Note': note
        })

    # Create a DataFrame from the results
    df_shapiro = pd.DataFrame(shapiro_results)
    return df_shapiro

In [18]:
def check_normality(df, alpha=0.05):
    df['Normality'] = df['p-value'].apply(
        lambda p: 'Normal' if p >= alpha else 'Not Normal'
    )
    return df   

In [19]:
tmp1 = check_normality(shapiro_test(clean_df, 'gpc_refno', 'emissions'), alpha=0.05)
tmp2 = check_normality(shapiro_test(clean_df, 'gpc_refno', 'log_emissions'), alpha=0.05)

In [20]:
# Assuming tmp1 and tmp2 are already generated DataFrames

# Add a suffix to differentiate columns
tmp1 = tmp1.add_suffix('_emissions')
tmp2 = tmp2.add_suffix('_log_emissions')

# Rename 'gpc_refno_emissions' and 'gpc_refno_log_emissions' back to 'gpc_refno' for merging
tmp1 = tmp1.rename(columns={'gpc_refno_emissions': 'gpc_refno'})
tmp2 = tmp2.rename(columns={'gpc_refno_log_emissions': 'gpc_refno'})

# Merge the two DataFrames on 'gpc_refno'
df_merged = pd.merge(tmp1, tmp2, on=['gpc_refno'], how='inner')

In [37]:
#df_merged.to_csv('normality_test.csv', index=False)

### Test: Stationary Energy

In [ ]:
se_df = clean_df[clean_df['gpc_refno'].str.startswith('I.')]

In [ ]:
# Analize the distribution of the emissions
fig = px.histogram(se_df, 
                   x='emissions', 
                   nbins=200, 
                   labels={'emissions': 'Emissions'}, 
                   template="plotly_white")

# Customize the layout
fig.update_layout(
    xaxis_title="Emissions",
    yaxis_title="Frequency",
    bargap=0.1  # Adjust the gap between bars
)

# Show the figure
fig.show()

Since emissions are log-normally distributed, apply a log transformation to stabilize variance and normalize the data

In [ ]:
fig = px.histogram(se_df, 
                   x='log_emissions', 
                   nbins=200, 
                   labels={'emissions': 'Emissions'}, 
                   template="plotly_white")

# Customize the layout
fig.update_layout(
    xaxis_title="Emissions",
    yaxis_title="Frequency",
    bargap=0.1  # Adjust the gap between bars
)

# Show the figure
fig.show()

**Correlation Coefficients and Distribution of Emissions**

Use of correlation coefficients to quantify the relationships between `emissions` and `other variables` (such as Population, GDP, and Area)

1. **Pearson Correlation Coefficient**: Measures the linear relationship between two continuous variables.
- Log Transformation is needed to stabilize variance and approximate normality
- Benefits: highly effective if the relationship is genuinely linear after transformation
2. **Spearman Rank Correlation Coefficient**: non-parametric measure that assesses the monotonic relationship between two variables by converting them to ranks
- No specific distribution required
- The relationship between variables is monotonic (either consistently increasing or decreasing), but not necessarily linear
- Less sensitive to outliers and non-linear relationships
- Captures any monotonic relationship, whether linear or not

In [ ]:
# Pearson Correlation on Log-Transformed Data
pearson_corr = se_df[['Log_Emissions', 'Population', 'GDP', 'Area']].corr(method='pearson')
spearman_corr = se_df[['emissions', 'Population', 'GDP', 'Area']].corr(method='spearman')

## Correlations

In [14]:
# main indicators to consider
variables = ['Population', 'GDP', 'Area']

In [15]:
# correlation calculation
correlation_results = []

grouped = clean_df.groupby('gpc_refno')

for subsector, group in grouped:
    # check if the group has enough data points
    if len(group) < 10:
        # skip subsectors with insufficient data
        continue
    
    for var in variables:
        # Pearson Correlation on Log-Transformed Emissions
        pearson_corr, pearson_p = pearsonr(group['log_emissions'], group[var])
        
        # Spearman Correlation on Raw Emissions
        spearman_corr, spearman_p = spearmanr(group['emissions'], group[var])
        
        correlation_results.append({
            'gpc_refno': subsector,
            'Variable': var,
            'Pearson_Correlation': pearson_corr,
            'Pearson_p-value': pearson_p,
            'Spearman_Correlation': spearman_corr,
            'Spearman_p-value': spearman_p
        })

corr_df = pd.DataFrame(correlation_results)

In [16]:
corr_df = corr_df.melt(
    id_vars=['gpc_refno', 'Variable'],
    value_vars=['Pearson_Correlation', 'Spearman_Correlation'],
    var_name='Correlation_Type',
    value_name='Correlation_Value'
)

# Clean up the Correlation_Type column
corr_df['Correlation_Type'] = corr_df['Correlation_Type'].str.replace('_Correlation', '')

In [17]:
def categorize_correlation(r):
    """
    Categorize the correlation coefficient based on absolute value.
    """
    abs_r = abs(r)
    if 0.00 <= abs_r < 0.10:
        return "Negligible or No Correlation"
    elif 0.10 <= abs_r < 0.30:
        return "Very Weak Correlation"
    elif 0.30 <= abs_r < 0.50:
        return "Weak Correlation"
    elif 0.50 <= abs_r < 0.70:
        return "Moderate Correlation"
    elif 0.70 <= abs_r < 0.90:
        return "Strong Correlation"
    elif 0.90 <= abs_r <= 1.00:
        return "Almost Perfect Correlation"
    else:
        return "Undefined"

In [18]:
# Apply the function to Pearson and Spearman correlations
corr_df['Correlation_Strength'] = corr_df['Correlation_Value'].apply(categorize_correlation)

In [28]:
#corr_df.to_csv('correlation_results.csv', index=False)

## Statistics

**Emissions**

In [21]:
df_c40_unique = clean_df_reformat.drop_duplicates(subset=['City', 'Country', 'Region', 'Boundary', 'Year_calendar', 'gpc_refno', 'emissions', 'units', 'gas'])

In [22]:
# Calculate descriptive statistics for each Subsector based on Emissions
descriptive_stats_emissions = df_c40_unique.groupby(['gpc_refno'])['emissions'].describe().reset_index()

In [24]:
descriptive_stats_emissions['IQR'] = descriptive_stats_emissions['75%'] - descriptive_stats_emissions['25%']

In [26]:
descriptive_stats_emissions['lower_bound'] = descriptive_stats_emissions['25%'] - 1.5 * descriptive_stats_emissions['IQR']
descriptive_stats_emissions['upper_bound'] = descriptive_stats_emissions['75%'] + 1.5 * descriptive_stats_emissions['IQR']

In [30]:
descriptive_stats_emissions_1 = descriptive_stats_emissions.melt(
    id_vars=['gpc_refno', 'count'],
    value_vars=['mean', 'std', 'min', '25%', '50%', '75%', 'max', 'IQR', 'lower_bound', 'upper_bound'],
    var_name='statistic_name',
    value_name='statistic_value'
)

In [31]:
descriptive_stats_emissions_1['Variable_name'] = 'emissions'

In [32]:
descriptive_stats_emissions_1.head()

,gpc_refno,count,statistic_name,statistic_value,Variable_name
0,I.1.1,313.0,mean,1.362332e+09,emissions
1,I.1.2,311.0,mean,1.612574e+09,emissions
2,I.1.3,161.0,mean,1.466990e+08,emissions
3,I.2.1,309.0,mean,8.161034e+08,emissions
4,I.2.2,313.0,mean,2.627769e+09,emissions


**Per variable**

In [33]:
clean_df_reformat['per_Variable'] = clean_df_reformat['emissions'] / clean_df_reformat['Variable_value']

In [34]:
# Calculate descriptive statistics for each Subsector based on the per_variable value
descriptive_stats_by_variable = clean_df_reformat.groupby(['gpc_refno', 'Variable_name'])['per_Variable'].describe().reset_index()

In [35]:
descriptive_stats_by_variable['IQR'] = descriptive_stats_by_variable['75%'] - descriptive_stats_by_variable['25%']

In [36]:
descriptive_stats_by_variable['lower_bound'] = descriptive_stats_by_variable['25%'] - 1.5 * descriptive_stats_by_variable['IQR']
descriptive_stats_by_variable['upper_bound'] = descriptive_stats_by_variable['75%'] + 1.5 * descriptive_stats_by_variable['IQR']

In [37]:
descriptive_stats_by_variable_1 = descriptive_stats_by_variable.melt(
    id_vars=['gpc_refno', 'Variable_name', 'count'],
    value_vars=['mean', 'std', 'min', '25%', '50%', '75%', 'max', 'IQR', 'lower_bound', 'upper_bound'],
    var_name='statistic_name',
    value_name='statistic_value'
)

In [38]:
stats_per_variable_final = pd.merge(
    descriptive_stats_by_variable_1,
    corr_df,
    left_on=['gpc_refno', 'Variable_name'],
    right_on=['gpc_refno', 'Variable'],
    how='inner'
)

In [39]:
stats_per_variable_final.drop(columns=['Variable'], inplace=True)

In [40]:
df_final = pd.merge(
    stats_per_variable_final, 
    descriptive_stats_emissions_1, 
    on=['gpc_refno', 'count', 'statistic_name', 'statistic_value','Variable_name'], 
    how='outer')

In [42]:
df_final.to_csv('benchmark_file.csv', index=False)

## Plots

### Notation Keys

In [12]:
# Filter for non-numeric emissions and calculate value counts
non_numeric_counts = df_c40[df_c40['emissions'].apply(is_non_numeric)]['emissions'].value_counts()

non_numeric_counts_df = non_numeric_counts.reset_index()
non_numeric_counts_df.columns = ['Notation Key', 'Count'] 

fig = px.bar(non_numeric_counts_df, 
             x='Notation Key', 
             y='Count', 
             labels={'Notation Key': 'Notation Key', 'Count': 'Count'},
             template="plotly_white")

fig.show()

### Correlations

In [26]:
grouped = clean_df.groupby('gpc_refno')

unique_subsectors = sorted(clean_df['gpc_refno'].unique())

# Create the subsector dropdown widget
subsector_dropdown = Dropdown(
    options=unique_subsectors,
    value=unique_subsectors[0],
    description='Subsector:',
    disabled=False,
)

# Create the variable dropdown widget
variable_dropdown = Dropdown(
    options=variables,
    value=variables[0],
    description='Variable:',
    disabled=False,
)

def plot_subsector_variable(subsector, variable):
    # Retrieve the data for the selected subsector
    group = grouped.get_group(subsector)
    
    plt.figure(figsize=(10, 6))
    
    # Scatter plot of the selected variable vs Log_Emissions
    sns.scatterplot(data=group, x=variable, y='Log_Emissions', alpha=0.5)
    
    # Regression line
    sns.regplot(data=group, x=variable, y='Log_Emissions', scatter=False, color='red')
    
    plt.title(f'Log_Emissions vs {variable} for Subsector {subsector}')
    plt.xlabel(variable)
    plt.ylabel('Log(Emissions)')
    plt.show()

# Use interact to link both dropdowns to the plotting function
interact(plot_subsector_variable, subsector=subsector_dropdown, variable=variable_dropdown)

interactive(children=(Dropdown(description='Subsector:', options=('I.1.1', 'I.1.2', 'I.1.3', 'I.2.1', 'I.2.2',…

<function __main__.plot_subsector_variable(subsector, variable)>

**Purpose of Correlation Analysis**

To determine how emissions in each subsector relate to key variables (Population, GDP, Area), which informs how to normalize emissions for benchmarking.

**In the Context of Benchmarking Objectives**
- Tailored Benchmarks: By aligning benchmarks with variables that strongly influence emissions, you ensure that comparisons are meaningful and account for key determinants
- Avoiding Misleading Comparisons: Using weakly correlated variables could result in benchmarks that `don't accurately reflect` the underlying dynamics, leading to ineffective assessments

**High Correlation**: 
- When a subsector shows a high correlation with a specific variable (e.g., Population), it indicates a strong relationship between that variable and emissions in that subsector.
- *Implication for Benchmarking*: A high correlation suggests that the variable can be a reliable factor to normalize emissions, facilitating meaningful comparisons across different cities or regions.

**Low Correlation**:
- When a subsector exhibits low correlation with the variables considered (e.g., GDP, Area), it implies that these factors do not strongly influence emissions in that subsector.
- *Implication for Benchmarking*: 
    - Low correlation indicates that normalizing emissions based on these variables may not yield accurate or meaningful benchmarks. It also suggests that other unexamined factors may be driving emissions in these subsectors.
    - Recognizing that certain subsectors do not correlate strongly with the primary variables (Population, GDP, Area) highlights the need for alternative factors or more nuanced models for accurate benchmarking.
    - If the variable doesn't influence emissions using it for benchmarking won't give you meaningful insights.

In [27]:
fig = go.Figure()

for var in corr_df['Variable'].unique():
    # Filter data by variable
    df_var = corr_df[corr_df['Variable'] == var]
    
    # Add trace for Pearson Correlation 
    fig.add_trace(go.Scatter(
        x=df_var['Subsector'],
        y=df_var['Pearson_Correlation'],
        mode='markers',
        name=f'{var} (Pearson)',
        marker=dict(symbol='circle'),
        visible=True  
    ))
    
    # Add trace for Spearman Correlation
    fig.add_trace(go.Scatter(
        x=df_var['Subsector'],
        y=df_var['Spearman_Correlation'],
        mode='markers',
        name=f'{var} (Spearman)',
        marker=dict(symbol='circle'),
        visible=False  
    ))

fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{'visible': [True if 'Pearson' in trace.name else False for trace in fig.data]}],
                    label="Pearson Correlation",
                    method="restyle"
                ),
                dict(
                    args=[{'visible': [True if 'Spearman' in trace.name else False for trace in fig.data]}],
                    label="Spearman Correlation",
                    method="restyle"
                )
            ]),
            direction="down",
            showactive=True,
            x=0.17,
            y=1.15,
            xanchor="left",
            yanchor="top"
        )
    ]
)

fig.update_layout(
    title="Correlation Plot by Subsector and Variable",
    xaxis_title="Subsector (GPC Reference)",
    yaxis_title="Correlation Value",
    legend_title="Variable",
    template="plotly_white"
)

fig.show()

In [28]:
color_map = {
    "Negligible or No Correlation": "#d3d3d3",    # Light Grey
    "Weak Correlation": "#ffcc00",                # Yellow
    "Moderate Correlation": "#ff9900",            # Orange
    "Strong Correlation": "#ff6600",              # Dark Orange
    "Very Strong Correlation": "#cc0000",         # Red
    "Almost Perfect Correlation": "#660000"       # Dark Red
}

# Assign colors
df_long['Color'] = df_long['Correlation_Strength'].map(color_map)

fig = px.bar(
    df_long,
    x='Subsector',
    y='Correlation_Strength',
    color='Correlation_Strength',
    facet_row='Correlation_Type',
    category_orders={'Correlation_Strength': [
        "Negligible or No Correlation",
        "Weak Correlation",
        "Moderate Correlation",
        "Strong Correlation",
        "Very Strong Correlation",
        "Almost Perfect Correlation"
    ]},
    labels={
        'Correlation_Strength': 'Correlation Strength',
        'Subsector': 'Subsector',
        'Correlation_Type': 'Correlation Type'
    },
    title='Correlation Strengths by Subsector and Variable',
    hover_data=['Variable']
)

fig.update_traces(marker=dict(color=df_long['Color']))

fig.update_layout(
    showlegend=False,
    height=800,
    bargap=0.2,
    bargroupgap=0.1
)

fig.show()